In [1]:
import requests

url = "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/recent"

headers = {
	"x-rapidapi-key": "7a79b931f4mshd2ae1a5c14b73dcp10dac1jsnd2e8e16a0f54",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

print(response.json())


{'typeMatches': [{'matchType': 'International', 'seriesMatches': [{'seriesAdWrapper': {'seriesId': 11253, 'seriesName': "ICC Men's T20 World Cup 2026", 'matches': [{'matchInfo': {'matchId': 139142, 'seriesId': 11253, 'seriesName': "ICC Men's T20 World Cup 2026", 'matchDesc': '20th Match, Group D', 'matchFormat': 'T20', 'startDate': '1770975000000', 'endDate': '1770987600000', 'state': 'Complete', 'status': 'United Arab Emirates won by 5 wkts', 'team1': {'teamId': 26, 'teamName': 'Canada', 'teamSName': 'CAN', 'imageId': 776227}, 'team2': {'teamId': 7, 'teamName': 'United Arab Emirates', 'teamSName': 'UAE', 'imageId': 776242}, 'venueInfo': {'id': 51, 'ground': 'Arun Jaitley Stadium', 'city': 'Delhi', 'timezone': '+05:30', 'latitude': '28.637891', 'longitude': '77.243073'}, 'currBatTeamId': 7, 'seriesStartDt': '1770422400000', 'seriesEndDt': '1773100800000', 'isTimeAnnounced': True, 'stateTitle': 'UAE Won'}, 'matchScore': {'team1Score': {'inngs1': {'inningsId': 1, 'runs': 150, 'wickets': 

In [2]:
rec_20_data = response.json()

In [3]:
matches = []

for tm in rec_20_data['typeMatches']:
    for sm in tm.get('seriesMatches', []):
        series = sm.get('seriesAdWrapper', {})
        for match in series.get('matches', []):
            info = match.get('matchInfo', {})
            
            if info.get('state') != 'Complete':
                continue  # only completed matches

            matches.append({
                'match_desc': info.get('matchDesc'),
                'team1': info['team1']['teamName'],
                'team2': info['team2']['teamName'],
                'status': info.get('status'),
                'venue': info['venueInfo']['ground'],
                'start_date': int(info.get('startDate', 0))
            })


In [4]:
def parse_result(status):
    if 'won by' not in status:
        return None, None, None

    winner = status.split(' won by ')[0]
    margin_part = status.split(' won by ')[1]

    if 'runs' in margin_part:
        return winner, margin_part.replace(' runs', ''), 'runs'
    if 'wkts' in margin_part or 'wickets' in margin_part:
        return winner, margin_part.replace(' wkts', '').replace(' wickets', ''), 'wickets'

    return winner, margin_part, 'other'


In [5]:
for m in matches:
    winner, margin, victory_type = parse_result(m['status'])
    m['winning_team'] = winner
    m['victory_margin'] = margin
    m['victory_type'] = victory_type


In [6]:
recent_20 = sorted(
    matches,
    key=lambda x: x['start_date'],
    reverse=True
)[:20]


In [7]:
import pandas as pd

df = pd.DataFrame(recent_20)[[
    'match_desc',
    'team1',
    'team2',
    'winning_team',
    'victory_margin',
    'victory_type',
    'venue'
]]

df


,match_desc,team1,team2,winning_team,victory_margin,victory_type,venue
0,"22nd Match, Group B",Ireland,Oman,Ireland,96,runs,Sinhalese Sports Club
1,"3rd Match, Group B",Thailand Women,Malaysia Women,Thailand Women,99,runs,Terdthai Cricket Ground
2,2nd T20I,Pakistan Women,South Africa Women,South Africa Women,6,wickets,Willowmoore Park
3,4th T20I,Qatar Women,Oman Women,Oman Women,8,wickets,West End Park International Cricket Stadium
4,"21st Match, Group A",United States of America,Netherlands,United States of America,93,runs,MA Chidambaram Stadium
5,"20th Match, Group D",Canada,United Arab Emirates,United Arab Emirates,5,wickets,Arun Jaitley Stadium
6,3rd T20I,Qatar,Bahrain,Qatar,31,runs,West End Park International Cricket Stadium
7,"2nd Match, Group A",India A Women,United Arab Emirates Women,United Arab Emirates Women,7,wickets,Terdthai Cricket Ground
8,"19th Match, Group B",Zimbabwe,Australia,Zimbabwe,23,runs,R.Premadasa Stadium
9,"1st Match, Group A",Pakistan A Women,Nepal Women,Pakistan A Women,30,runs,Terdthai Cricket Ground
